# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../weather/city_weather_csv"
city_weather = pd.read_csv(csv_path)
city_weather =city_weather.drop(["Unnamed: 0"], axis = 1)
city_weather

,City,lat,lng,max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,aqtobe,50.2797,57.2072,9.53,55.0,5.0,2.68,KZ,1.619712e+09
1,burnie,-41.0667,145.9167,12.22,92.0,95.0,0.45,AU,1.619712e+09
2,ribeira grande,38.5167,-28.7000,18.00,52.0,40.0,4.63,PT,1.619712e+09
3,shingu,33.7333,135.9833,12.33,88.0,93.0,3.01,JP,1.619712e+09
4,shevchenkove,49.6959,37.1735,16.67,27.0,0.0,4.00,UA,1.619712e+09
...,...,...,...,...,...,...,...,...,...
905,fairbanks,64.8378,-147.7164,3.33,74.0,75.0,1.54,US,1.619712e+09
906,ugoofaaru,5.6667,73.0000,28.92,71.0,84.0,3.21,MV,1.619712e+09
907,diego de almagro,-26.3667,-70.0500,23.10,28.0,42.0,3.94,CL,1.619712e+09
908,turnu rosu,45.6333,24.3000,18.00,63.0,75.0,1.54,RO,1.619712e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locales = city_weather[['lat','lng']]
humidity = city_weather['Humidity'].astype(float)

In [4]:
vacay_map = gmaps.figure()

heatmap = gmaps.heatmap_layer(locales, weights=humidity, max_intensity=100, dissipating = False,
                                 point_radius = 1.5)
vacay_map.add_layer(heatmap)

vacay_map


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_cities = city_weather[(city_weather['max temp'] <5) &
                               (city_weather['Wind Speed'] < 4) &
                              (city_weather['Cloudiness'] < 10)]

vacation_cities =vacation_cities.dropna()
vacation_cities

,City,lat,lng,max temp,Humidity,Cloudiness,Wind Speed,Country,Date
240,arman,59.7000,150.1667,-2.51,89.0,9.0,2.38,RU,1.619712e+09
346,punta arenas,-53.1500,-70.9167,4.00,81.0,0.0,3.60,CL,1.619712e+09
569,kedrovyy,56.1667,91.8167,3.00,87.0,0.0,2.00,RU,1.619712e+09
763,deputatskiy,69.3000,139.9000,-14.79,88.0,5.0,1.22,RU,1.619712e+09
800,anadyr,64.7500,177.4833,-9.00,86.0,0.0,2.00,RU,1.619712e+09
867,nizhneangarsk,55.7833,109.5500,-5.72,77.0,7.0,2.72,RU,1.619712e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_cities
hotel_df['Hotel Name'] =  ' '

hotel_df
x = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
for index, row in hotel_df.iterrows():
    lat = row['lat']
    long = row["lng"]
    
    params['location'] = f'{lat},{long}'
    
    try:
        hotels = requests.get(base_url, params = params).json()
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        
    except:
        KeyError
        hotel_df.loc[index, "Hotel Name"] = None

In [7]:
hotel_df = hotel_df.dropna()

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content = hotel_info)

vacay_map.add_layer(markers)
vacay_map



# Display figure


Figure(layout=FigureLayout(height='420px'))